## Import

In [ ]:
import random
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
import librosa

from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/음성 분류

/content/drive/MyDrive/음성 분류


## Hyperparameter Setting

In [ ]:
CFG = {
    'SR':16000,
    'N_MFCC':32, # Melspectrogram 벡터를 추출할 개수
    'SEED':42
}

## Fixed Random-Seed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-Processing

In [ ]:
train_df_sst = pd.read_csv('./train_df_sst.csv')
test_df_sst = pd.read_csv('./test_df_sst.csv')

In [ ]:
train_df_sst.head()

,id,path,label,transcription
0,TRAIN_0000,./train/TRAIN_0000.wav,1,IT'S A AT A COCK
1,TRAIN_0001,./train/TRAIN_0001.wav,2,THE SERVICE IS SLICK
2,TRAIN_0002,./train/TRAIN_0002.wav,4,WE'LL STOP IN A COUPLE OF MINUTES
3,TRAIN_0003,./train/TRAIN_0003.wav,5,MAYBE TO MORROW IT WILL BE COLD
4,TRAIN_0004,./train/TRAIN_0004.wav,4,DON'T FORGET A JACKET


In [ ]:
test_df_sst.head()

,id,path,transcription
0,TEST_0000,./test/TEST_0000.wav,I WOULD LIKE A NEW ALARM CLOCK
1,TEST_0001,./test/TEST_0001.wav,MAYBE TO MORROW IT WILL BE COLD
2,TEST_0002,./test/TEST_0002.wav,MAYBE TOMORROW IT WILL BE COLD
3,TEST_0003,./test/TEST_0003.wav,IT'S ELEVEN O'CLOCK
4,TEST_0004,./test/TEST_0004.wav,DON'T FORGET A JACKET


In [ ]:
# def get_mfcc_feature(df):
#     features = []
#     for path in tqdm(df['path']):
#         # librosa패키지를 사용하여 wav 파일 load
#         y, sr = librosa.load(path, sr=CFG['SR'])
#         # librosa패키지를 사용하여 mfcc 추출
#         mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])
#         y_feature = []
#         # 추출된 MFCC들의 평균을 Feature로 사용
#         for e in mfcc:
#             y_feature.append(np.mean(e))
#         features.append(y_feature)

#     mfcc_df = pd.DataFrame(features, columns=['mfcc_'+str(x) for x in range(1,CFG['N_MFCC']+1)])
#     return mfcc_df

In [ ]:
# train_x = get_mfcc_feature(train_df)
# test_x = get_mfcc_feature(test_df)

In [ ]:
# train_y = train_df['label']

## KoBERT load

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm 
!pip install sentencepiece
!pip install torch
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-d2wzagan/kobert-tokenizer_409c8a2c53ff46fb99f1a2812290ac99
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-d2wzagan/kobert-tokenizer_409c8a2c53ff46fb99f1a2812290ac99
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-5zg25vv7/kobert-tokenizer_f9d7befa83be4f70af818f6cfd227e13
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-5zg25vv7/kobert-tokenizer_f9d7befa83be4f70af818f6cfd227e13
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [ ]:
from kobert_tokenizer import KoBERTTokenizer

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [ ]:
import torch
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
out = model(input_ids = torch.tensor(inputs['input_ids']),
              attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape
torch.Size([1, 768])

torch.Size([1, 768])

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
model = AutoModel.from_pretrained("skt/kobert-base-v1")

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
!apt-get install -y gluonnlp
!apt-get install -y mxnet

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package gluonnlp
Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package mxnet


In [ ]:
import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:2.0.1+cu118
cuda version: 11.8
cudnn version:8700


In [ ]:
# gluonnlp instakk
!pip install --upgrade mxnet>=1.6.0
!pip install gluonnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gluonnlp==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gluonnlp as nlp

ImportError: ignored

## Submission

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission['label'] = preds
submission.to_csv('./baseline_submission.csv', index=False)